# Run Partial Matching Baseline

This notebook will run alignment algorithms on clean to modified audio

In [1]:
import numpy as np
import numba
import time
import librosa as lb
import multiprocessing
import subprocess
import pickle
import logging
import sys
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os.path
from pathlib import Path
from align_algs import *

In [ ]:
user = 'cchang'

In [2]:
hop_sec = 512 * 1 / 22050
n_cores = 8
downsample = 1

In [3]:
times = np.array([5,10,20,30])
featdir1_root = Path('/home/{}/ttmp/features/partial_match'.format(user))
featdir2 = Path('/home/{}/ttmp/features/clean'.format(user))

In [4]:
# Set which cfg file we want to use for our tests
cfg_options = ['toy', 'small', 'small2', 'medium', 'train_benchmark', 'test_benchmark']
cfg = cfg_options[5]

if cfg == 'toy':
    query_list = 'cfg_files/query.train_toy.list'
elif cfg == 'small':
    query_list = 'cfg_files/query.train_small.list'
elif cfg == 'small2':
    query_list = 'cfg_files/query.train_small2.list'
elif cfg == 'medium':
    query_list = 'cfg_files/query.train_medium.list'
elif cfg == 'train_benchmark':
    query_list = 'cfg_files/query.train.list'
elif cfg == 'test_benchmark':
    query_list = 'cfg_files/query.test.list'

## Calculate Alignments

In [ ]:
# For DTW Baselines
folder_names = ['dtw233', 'dtw111']

for t in times:
    EXPERIMENTS_ROOT = '/home/%s/ttmp/partial_match/%s/experiments/align_clean_to_%ss' % (user, cfg, t)
    
    for i in range(1, len(folder_names) + 1):
        outdir = Path(EXPERIMENTS_ROOT + "/" + folder_names[i-1])
        featdir1 = featdir1_root / ('partial_match_%ss' % t)
        inputs = align_batch(i, query_list, featdir1, featdir2, outdir, n_cores, downsample)

In [ ]:
# For NWTW
gammas = [0.2, 1]
folder_names = ['nwtw/gamma_%s' %g for g in gammas]

for t in times:
    EXPERIMENTS_ROOT = '/home/%s/ttmp/partial_match/%s/experiments/align_clean_to_%ss' % (user, cfg, t)
    
    for i in range(len(gammas)):
        outdir = Path(EXPERIMENTS_ROOT + "/" + folder_names[i])
        featdir1 = featdir1_root / ('partial_match_%ss' % t)
#         inputs = align_batch(i, query_list, featdir1, featdir2, outdir, n_cores, downsample)
        alignNW_batch(query_list, featdir1, featdir2, outdir, n_cores, downsample, gammas[i])

In [5]:
gamma = 10
beta = 150
# gammas = [1,5,10,20,50]
# betas = [0,10,100,150,200,250]

steps = np.array([1,1,2,1,1,2]).reshape((-1,2))
weights = np.array([2,3,3])

# folder_names = ['hstw/gamma_%s_beta_150' %g for g in gammas]
# folder_names = ['hstw/gamma_10_beta_%s' % b for b in betas]
folder_names = ['hstw/gamma_10_beta_150']

In [6]:
folder_names

['hstw/gamma_10_beta_0',
 'hstw/gamma_10_beta_10',
 'hstw/gamma_10_beta_100',
 'hstw/gamma_10_beta_150',
 'hstw/gamma_10_beta_200',
 'hstw/gamma_10_beta_250']

In [ ]:
# For HSTW
for t in times:
    EXPERIMENTS_ROOT = '/home/%s/ttmp/partial_match/%s/experiments/align_clean_to_%ss' % (user, cfg, t)
    
    for i in range(len(folder_names)):
        outdir = Path(EXPERIMENTS_ROOT + "/" + folder_names[i])
        featdir1 = featdir1_root / ('partial_match_%ss' % t)
        align_HPTW_batch(query_list, featdir1, featdir2, outdir, n_cores, steps, weights, gamma, betas[i], downsample)